# Kак роботы планируют траекторию движения

In [16]:
!pip install pygame

You are using pip version 10.0.1, however version 20.3.4 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


![1](./Image/1.png)

В этом ноутбуке мы рассмотрим проблему нахождения кратчайшего пути на карте. Эта задача используется в робототехнике, компьютерных играх и сервисах такси, доставки и т.п. В данном ноутбуке карта будет представлять из себя сетку(grid), в которой одни клетки проходимы, а другие представляют собой препятствия. Существуют разные спрособы определять соседей клетки, например, соседями могут являться только смежные клетки с данной или можно еще разрешать ходить по диагонали при некоторых ограничениях. В этом задании будем предполагать, что мы можем ходить только по смежным клеткам: вверх, вниз, влево и вправо.

Один из алгоритмов нахождения кратчайшего пути - A* (A star). Процесс нахождения пути выглядит так:

![2](./Image/a_star_pathfinding.gif)

Ваша задача - реализовать алгоритм A*. Запускайте клетки ноутбука по очереди, дополняя пропуски своим кодом. Удачи!

После написания всего кода вы сможете запустить его как приложение и посмотреть на его работу вживую, поэтому не пугайтесь большому количеству кода, весь код понимать не надо, а в нужных местах будут пояснения :)

In [2]:
import pygame
import math

"""
Это просто набор констант для приложения (цвета, число клеток в окне и прочее)
"""
WIDTH = 1000#800
pygame.init()
WIN = pygame.display.set_mode((WIDTH,WIDTH))
pygame.display.set_caption("A* Path finding Algorithm")
RED = (255,0,0)
GREEN = (0,255,0)
BLUE = (0,0,255)
YELLOW = (255,255,0)
WHITE = (255,255,255)
BLACK = (0,0,0)
PURPLE = (128,0,128)
ORANGE = (255,165,0)
GREY = (128,128,128)
TURQUOISE = (64,224,208)

# EPS может понадобиться, чтобы проверять равенство двух вещественных чисел с точностью до какого-то знака после запятой
EPS = 1e-8

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


Класс Spot представляет собой клетку на игровом поле, где

- row, col - ряд и колонка клетки (координаты). Они вам понадобятся.
- x, y - это пиксельные координаты, каждая клетка имеет ширину width, и это левый верхний угол клетки, он просто нужен, чтобы нарисовать клетку с помощью библиотеки pygame. Это вам не нужно.
- color - цвет клетки, в коде ниже написано, что означает каждый из цветов. Вам понадобится правильно раскрашивать клетки в вашем алгоритме.
- neighbors - 4 соседа Spot'a (тоже представители класса Spot)
- width - ширина клетки на поле (в пикселях)
- total_rows - общее число рядов И колонок (поле квадратное)

In [3]:
#Spot represent individual points in the grid
class Spot:
    def __init__(self, row, col, width, total_rows):
        self.row = row
        self.col = col
        self.x = row * width
        self.y = col * width
        self.color = WHITE
        self.neighbors = []
        self.width = width
        self.total_rows = total_rows
        
    def __eq__(self, other):
        if not isinstance(other, Spot):
            return False
        return (self.row == other.row) and (self.col == other.col)

    def get_pos(self):
        return self.row, self.col
    
    # клетка(вершина) лежит в CLOSED
    def is_closed(self):
        return self.color == RED
    
    # клетка(вершина) лежит в OPEN
    def is_open(self):
        return self.color == GREEN

    # клетка - препятствие
    def is_barrier(self):
        return self.color == BLACK

    # стартовая клетка
    def is__start(self):
        return self.color == ORANGE

    # целевая клетка
    def is_end(self):
        return self.color == PURPLE

    # обычная клетка
    def reset(self):
        self.color = WHITE

    # перекрасить клетку, если она попала в CLOSED
    def make_closed(self):
        self.color = RED

    def make_start(self):
        self.color = ORANGE

    # перекрасить клетку, если она попала в OPEN
    def make_open(self):
        self.color = GREEN

    def make_barrier(self):
        self.color = BLACK
    
    def make_end(self):
        self.color = TURQUOISE
    
    # клетка найденного пути
    def make_path(self):
        self.color = PURPLE
    
    def draw(self, win):
        pygame.draw.rect(win, self.color, (self.x, self.y, self.width, self.width))

    def update_neighbors(self, grid):
        self.neighbors=[]
        if self.row + 1 < self.total_rows and not grid[self.row + 1][self.col].is_barrier():
            self.neighbors.append(grid[self.row + 1][self.col])
    
        if self.row > 0 and not grid[self.row - 1][self.col].is_barrier():
            self.neighbors.append(grid[self.row - 1][self.col])

        if self.col < self.total_rows - 1 and not grid[self.row][self.col + 1].is_barrier():
            self.neighbors.append(grid[self.row][self.col + 1])

        if self.col > 0 and not grid[self.row][self.col - 1].is_barrier():
            self.neighbors.append(grid[self.row][self.col - 1])

Класс Spot будет представлять клетки карты, но для алгоритма нам нужен набор вершин Node со своими полями:

- i, j - координаты вершины (сопоставляются с Spot с соответствующим row, col)
- g - g-value
- F - F-value
- parent - вершина, из которой мы пришли в данную. По этому полю мы сможем восстановить путь.

In [17]:
class Node:
    def __init__(self, i, j, g = math.inf, h = math.inf, F = None, parent = None):
        self.i = i
        self.j = j
        self.g = g
        if F is None:
            self.F = self.g + h
        else:
            self.F = F        
        self.parent = parent
    
    def __eq__(self, other):
        return (self.i == other.i) and (self.j == other.j)
    
    def __gt__(self, other):
        return self.F - EPS > other.F
    
    def __hash__(self):
        return 31 + 7 * (hash(self.i) + 7 * hash(self.j))
    
    def __str__(self):
        return 'i: {0}, j: {1}, g: {2}, F: {3}, parent: {4}'.format(self.i, self.j, self.g, self.F, self.parent)

В A\*, в отличие от алгоритма Дейкстры, мы используем метрики, чтобы отсеять заведомо неправильные вершины. Например, если мы хотим долететь из Новосибирска до Владивостока, то зачем лететь с пересадкой в Москве, ведь она находится в противоположной стороне. Хорошая метрика для A\* как раз поможет избежать лишних действий.

In [5]:
def ManhattanMetric(x1, y1, x2, y2):
    return abs(x1 - x2) + abs(y1 - y2)

In [6]:
# Функция для воостановления пути
def reconstruct_path(currentNode, grid, draw):
    while currentNode.parent is not None:
        currentNode = currentNode.parent
        currentSpot = grid[currentNode.i][currentNode.j]
        currentSpot.make_path()
        # отрисовываем вершины итеративно, чтобы путь строился динамически
        draw()
    currentSpot = grid[currentNode.i][currentNode.j]
    currentSpot.make_start()
    draw()

### Реализация OPEN и CLOSED

Эффективная реализация OPEN и CLOSED очень важна для любого алгоритма поиска. Ниже вы можете увидеть базовую реализацию этих классов, но с ними алгоритм поиска будет работать на больших картах очень долго (минуты и часы), поэтому вам нужно написать свою версию, которая будет выполнять операции с этими множествами более эффективно.

In [7]:
class OpenBase:
    def __init__(self):
        pass
    
    def __len__(self):
        pass
    
    def __iter__(self):
        pass

    # isEmpty должна сообщать о том, пустое множество или нет
    def isEmpty(self):
        pass

    # AddNode это метод добавляющий или обновляющий вершину в OPEN
    # Не забудьте обработать следующие случаи:
    # - вершина уже в OPEN но новое g-value лучше;
    # - вершина уже в OPEN но новое g-value хуже;
    # - вершина не в OPEN.
    def AddNode(self, item: Node):
        pass

    # GetBestNode это метод, возвращающий лучшую вершину из OPEN (с наименьшим f-value) и удаляющий ее из OPEN
    def GetBestNode(self):
        pass

In [8]:
class OpenList(OpenBase):
    def __init__(self):
        self.elements = []
    
    def __iter__(self):
        return iter(self.elements)

    def __len__(self):
        return len(self.elements)

    def isEmpty(self):
        if len(self.elements) != 0:
            return False
        return True

    def GetBestNode(self):
        bestF = math.inf
        bestCoord = 0
        for i in range(len(self.elements)):
            if (self.elements[i].F < bestF) or (abs(self.elements[i].F - bestF) < EPS):
                bestCoord = i
                bestF = self.elements[i].F
                
        best = self.elements.pop(bestCoord)
        return best
    

    def AddNode(self, item):
        for coord in range(len(self.elements)):
            if self.elements[coord].i == item.i and self.elements[coord].j == item.j:
                if (self.elements[coord].g > item.g) or (abs(self.elements[coord].g - item.g) < EPS):
                    self.elements[coord].F = item.F
                    self.elements[coord].g = item.g
                    self.elements[coord].parent = item.parent
                    return
                else:
                    return
        self.elements.append(item)
        return

От OPEN требуется выполнять следующие операции: добавлять вершину (обновлять ее значение) и быстро доставать ноду с минимальным F-value. Какие структуры данных могут эффективно выполнять эти операции?

In [9]:
import heapq

class YourOpen(OpenBase):
    def __init__(self):
        self.elements = []
        self.in_open = set()
    
    def __iter__(self):
        return iter(sorted(list([n for n in self.elements if n in self.in_open])))

    def __len__(self):
        return len(self.in_open)

    def isEmpty(self):
        return len(self.elements) == 0
    
    def GetBestNode(self):
        min_node = heapq.heappop(self.elements)
        self.in_open.remove(min_node)
        while len(self.elements) != 0 and self.elements[0] not in self.in_open:
            heapq.heappop(self.elements)
        return min_node

    def AddNode(self, item: Node):
        self.in_open.add(item)
        heapq.heappush(self.elements, item)

In [10]:
class ClosedBase:

    def __init__(self):
        pass

    def __iter__(self):
        pass
    
    def __len__(self):
        pass
    
    
    def AddNode(self, item: Node, *args):
        pass

    def WasExpanded(self, item: Node, *args):
        pass

In [11]:
class ClosedList (ClosedBase):
    
    def __init__(self):
        self.elements = []

    def __iter__(self):
        return iter(self.elements)
    
    def __len__(self):
        return len(self.elements)
    
    # AddNode is the method that inserts the node to CLOSED
    def AddNode(self, item, *args):
        self.elements.append(item)

    # WasExpanded is the method that checks if a node has been expanded
    def WasExpanded(self, item, *args):
        return item in self.elements

От CLOSED требуется выполнять операции добавления и узнавать, есть ли элемент в множестве.

In [12]:
class YourClosed (ClosedBase):
    
    def __init__(self):
        self.elements = set()

    def __iter__(self):
        return iter(self.elements)
    
    def __len__(self):
        return len(self.elements)
    
    # AddNode is the method that inserts the node to CLOSED
    def AddNode(self, item : Node, *args):
        self.elements.add(item)

    # WasExpanded is the method that checks if a node has been expanded
    def WasExpanded(self, item : Node, *args):
        return item in self.elements

# A* Algorithm Implementation

In [13]:
#A* Algorithm Implementation
def algorithm(draw, grid, start, end, h=ManhattanMetric):
    open_set = OpenList()
    closed_set = ClosedList()
    startNode = Node(start.row, start.col, 0, h(start.row, start.col, end.row, end.col))
    open_set.AddNode(startNode)

    while not open_set.isEmpty():
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()

        currentNode = open_set.GetBestNode()
        currentSpot = grid[currentNode.i][currentNode.j]
        
        closed_set.AddNode(currentNode)
        if currentSpot != start:
            currentSpot.make_closed()
        if currentSpot == end:
            reconstruct_path(currentNode, grid, draw)
            end.make_end()
            return True

        g_val = currentNode.g
        for neighbor in currentSpot.neighbors:
            if not closed_set.WasExpanded(Node(neighbor.row, neighbor.col)):
                newNode = Node(neighbor.row, neighbor.col, g_val + 1, h(neighbor.row, neighbor.col, end.row, end.col), None, currentNode)
                open_set.AddNode(newNode)
                neighbor.make_open()
                
            draw()
            
    return False

Это сама программа, тут ничего не надо трогать!

In [15]:
#Implenents Grid
def make_grid(rows,width):
    grid=[]
    gap=width//rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            spot=Spot(i,j,gap,rows)
            grid[i].append(spot)
    return grid


#Draws Text on the left-Hand corner
def draw_text(Win,rows,width):
    gap=width//rows
    font = pygame.font.Font('freesansbold.ttf',20 )
    text = font.render("A* Path Finding Visualizer",True,WHITE,BLACK)
    textRect=text.get_rect()
    textRect.center=(rows//2+83-gap*2,rows//2)
    Win.blit(text, textRect)

#Draws Grid
def draw_grid(win,rows,width):
    gap=width//rows
    for i in range(rows):
        pygame.draw.line(win,GREY,(0,i*gap),(width,i*gap))
    for j in range(rows):
        pygame.draw.line(win,GREY,(j*gap,0),(j*gap,width))


#Draws the screen and everything in it.
def draw(win,grid,rows,width):
    win.fill(WHITE)
    for row in grid:
        for spot in row:
            spot.draw(win)
    
    draw_grid(win,rows,width)
    draw_text(win,rows,width)
    pygame.display.update()

#To get position of the mouse click on the screen
def get_clicked_pos(pos,rows,width):
    gap=width//rows 
    y,x = pos
    row=y//gap
    col=x//gap
    return row,col 

#Main function which executes everything thing
def main(win,width):
    ROWS = 50
    grid = make_grid(ROWS,width)
    start = None
    end = None
    run = True
    started = False
    print("Here:{}".format(run  ))
    while(run):
        draw(win,grid,ROWS,WIDTH)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run=False

        
            if pygame.mouse.get_pressed()[0]:#LEFT
                pos = pygame.mouse.get_pos()
                row, col = get_clicked_pos(pos, ROWS, width)
                spot = grid[row][col]
                if not start and spot != end:
                    start = spot
                    start.make_start() 
                elif not end and spot != start:
                    end = spot
                    end.make_end()
                elif spot != end and spot != start:
                    #print("making barrier")
                    spot.make_barrier()

            elif pygame.mouse.get_pressed()[2]: # RIGHT
                pos=pygame.mouse.get_pos()
                row,col=get_clicked_pos(pos,ROWS,width)
                spot=grid[row][col]
                spot.reset()

                if spot == start:
                    start=None
                
                if spot == end:
                    end=None
                    
            if event.type==pygame.KEYDOWN:
                if event.key==pygame.K_SPACE and start and end:
                    
                    for row in grid:
                        #print("row:{}".format(len(row)))
                        for spot in row:
                            spot.update_neighbors(grid)
                    algorithm(lambda:draw(win,grid,ROWS,width),grid,start,end)
                     
                if event.key == pygame.K_c:
                    start = None
                    end = None
                    grid = make_grid(ROWS,width)
                

    pygame.quit()
        
main(WIN,WIDTH)

Here:True


error: display Surface quit